Created on Wednesday 06 January 2021  

**Group 2 - Identification**

**"Journal du net" scraper**

@authors: Marianne Manson

## URL

Article link : https://www.journaldunet.fr/management/guide-du-management/1204934-management-operationnel/

__Lists:__ 

By number of pages
- https://www.journaldunet.com/cybersecurite/list?page=1
- https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1
- https://www.journaldunet.com/e-rh/list?page=1
- https://www.journaldunet.com/solutions/analytics/big-data/list?page=1
- others

By year
- https://www.journaldunet.com/management/list/2021-1-1/
- others


In [34]:
#lib import
import pandas as pd
import numpy as np
import re
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
df_scrap = pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','art_img','art_auth','art_tag']  )
df_scrap

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag


In [11]:
def get_nb_pages(first_page_url) -> int:
    '''Documentation
    
    Parameters:
        first_page_url: url of the first page of the article list
    
    Out:
        nb_pages: number of pages in the article list
    '''
    req = get(first_page_url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    div_num_page = html_soup.find("section",{'class':"ccmcss_paginator ccmcss_paginator--page"})
    div_num_page = div_num_page.find("ul")
    list_link_page = div_num_page.find_all("a")
    nb_pages = int(str(list_link_page[-1].text))
    return nb_pages

In [20]:
print(get_nb_pages("https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1"))

109


In [18]:
def get_url_page_jdn_nb_pages(first_page):
    """Documentation
    
    Parameters:
        first_page: url of first the page where articles are listed
    
    Out:
        url_page_list: list with the urls of the pages where articles are listed
    """
    url_page_list = [first_page]
    nb_pages = get_nb_pages(first_page)
    for num_page in range(2,nb_pages+1):
        url_page_list.append(first_page[:-1] + str(num_page))
    return url_page_list

In [21]:
test_list = get_url_page_jdn_nb_pages("https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1")
test_list

['https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=2',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=3',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=4',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=5',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=6',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=7',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=8',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=9',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=10',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=11',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=12',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=13',
 'https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=14',
 'https://www.j

In [6]:
def get_url_article_jdn(url_page):
    """Documentation
    
    Parameters:
        url_page: url of the page where articles are listed
    
    Out:
        url_art_list: list with the urls of articles
    """
    url_art_list = []
    req = get(url_page)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    art_divs = html_soup.find_all("h4")
    for div in art_divs:
        url = div.find("a")['href']
        url_art_list.append(url)
    return url_art_list

In [7]:
test_list = get_url_article_jdn("https://www.journaldunet.com/cybersecurite/list?page=1")
test_list

['https://www.journaldunet.com/solutions/dsi/1496707-stanislas-de-bentzmann-devoteam/',
 'https://www.journaldunet.com/solutions/dsi/1496655-la-cybersecurite-est-une-question-informationnelle/',
 'https://www.journaldunet.com/solutions/dsi/1496653-la-securite-du-cloud-post-pandemie-doit-demarrer-a-l-edge-avec-l-hyper-automatisation/',
 'https://www.journaldunet.com/web-tech/cloud/1496645-les-10-previsions-technologiques-cloud-et-cybersecurite-pour-2021/',
 'https://www.journaldunet.com/solutions/dsi/1496537-les-pme-et-les-tpe-doivent-s-armer-pour-affronter-les-ransomwares-as-a-service/',
 'https://www.journaldunet.com/solutions/dsi/1496549-le-pouvoir-du-stockage-a-l-ere-de-l-industrie-4-0/',
 'https://www.journaldunet.com/solutions/dsi/1496567-la-protection-des-donnees-un-sujet-prioritaire-par-ces-temps-incertains-3-tendances-qui-devraient-rythmer-l-annee-2021/',
 'https://www.journaldunet.com/solutions/dsi/1496571-e-commercants-pas-de-joyeux-noel-sans-cybersecurite/',
 'https://www.jo

In [39]:
def scraping(url):
    '''Documentation
    
    Parameters:
        url: url of the scraped page
    
    Out:
        new_row: data to put in dataframe
    '''
    req = get(url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    # content_html, content (maybe clean a little the content)
    try:
        content_html = html_soup.find("div",{'id':"jArticleInside"})
        content = content_html.text
    except:
        content_html = np.nan
        content = np.nan
    #date
    try:
        date = html_soup.find("time",{'itemprop':"publishDate"})['datetime']
        format_end = date[-5:]
        date = datetime.strptime(date,'%Y-%m-%dT%H:%M:%S+'+format_end)
        date = date.strftime('%Y-%m-%d')
        #possibly change where the date is extracted
        #see <script type="application/ld+json"> 
    except:
        date = datetime.today().strftime('%Y-%m-%d')
    #title
    try:
        zone_title = html_soup.find("div",{'id':"jStickySize"})
        title = zone_title.find("h1")
        title = title.text
    except:
        title = "no_data"
    #img
    try:
        zone_img = content_html.find("p",{'class':"app_entry_lead"})
        img = zone_img.find("img")['src']
    except:
        img = "no_data"
    #author
    try:
        link_author = html_soup.find("a",{'rel':"author"})
        author = link_author.text
    except:
        author = "no_data"
    #tags
    head = html_soup.find("head")
    scripts_list = head.find_all("script")
    script = str(scripts_list[1])
    pattern = re.compile("keywords: \[(\"(\w|\-|\d)*\",?)*\]")
    match =  re.search(pattern, script)
    list_tag_str = match.group(0)
    list_tag_str = list_tag_str[11:-1]
    list_tag_str = list_tag_str.replace("-"," ")
    list_tag = list_tag_str.split(",")
    #data 
    new_row = {'art_content': content ,
               'art_content_html': content_html ,
               'art_published_datetime': date ,
               'art_lang': 'fr' , 
               'art_title' : title , 
               'art_url' : url ,
               'src_name' :'journal du net'  ,
               'src_type' : 'xpath_source' ,
               'src_url' : 'https://www.journaldunet.com/',
               'art_img' : img ,
               'art_auth': author, # No author specified
               'art_tag': list_tag}
    return new_row

In [40]:
new_row = scraping("https://www.journaldunet.com/solutions/dsi/1496655-la-cybersecurite-est-une-question-informationnelle/")
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,\n\n Les remparts de la sécurité IT...,"[\n, [\n Les remparts de la sécurit...",2021-01-05,fr,\n La cybersécu...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Benjamin De Rose,"[""controller content"", ""cms journaldunet 14966..."


In [41]:
# Test on https://www.journaldunet.com/solutions/analytics/big-data/list?page=1
url_pages = get_url_page_jdn_nb_pages("https://www.journaldunet.com/solutions/analytics/big-data/list?page=1")
url_articles = []
for page in url_pages:
    url_articles.extend(get_url_article_jdn(page))
df_test = df_scrap
for article in url_articles:
    print(article)
    new_row = scraping(article)
    df_test = df_test.append(new_row, ignore_index=True)
df_test

https://www.journaldunet.com/economie/finance/1496045-les-graphes-dans-l-assurance-les-liaisons-vertueuses/
https://www.journaldunet.com/solutions/dsi/1495627-la-data-responsable-nouvel-accelerateur-de-l-entreprise/
https://www.journaldunet.com/solutions/dsi/1495193-la-simplification-de-la-gestion-des-bases-de-donnees-libere-l-agilite-de-l-entreprise/
https://www.journaldunet.com/solutions/dsi/1495011-maitrisez-les-donnees-clients-entreprises-fournisseurs-une-aide-precieuse-dans-la-recherche-de-rentabilite-et-de-productivite/
https://www.journaldunet.com/solutions/dsi/1494903-la-dette-liee-a-la-donnee-comme-frein-aux-nouvelles-initiatives/
https://www.journaldunet.com/solutions/dsi/1494825-comment-reconcilier-big-data-et-connaissance-client/
https://www.journaldunet.com/solutions/dsi/1494747-quand-les-exceptions-sont-la-regle-les-donnees-et-la-planification-sont-la-cle/
https://www.journaldunet.com/ebusiness/crm-marketing/1494563-industrie-du-sport-et-crise-sanitaire-sortir-de-la-crise

https://www.journaldunet.com/ebusiness/crm-marketing/1486877-le-marketing-digital-est-mort/
https://www.journaldunet.com/solutions/dsi/1486867-quand-l-art-s-empare-des-donnees/
https://www.journaldunet.com/solutions/dsi/1486620-cas-client-edelia-du-management-agile-a-la-certification/
https://www.journaldunet.com/management/direction-generale/1486299-placer-l-humain-au-coeur-de-la-transformation-numerique/
https://www.journaldunet.com/ebusiness/crm-marketing/1486199-la-segmentation-clients-par-les-donnees-5-enseignements-terrain/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1486131-un-nouvel-outil-pour-les-pirates-informatiques-l-ia-dans-la-cybersecurite/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1459422-l-ia-pour-reconcilier-la-gestion-de-la-performance-des-reseaux-et-des-applications/
https://www.journaldunet.com/ebusiness/commerce/1446878-qui-pourra-detroner-amazon/
https://www.journaldunet.com/solutions/dsi/1445795-redis-labs-s-erige-en-

https://www.journaldunet.com/solutions/dsi/1207794-trophees-big-data-paris-2018-voici-les-gagnants/
https://www.journaldunet.com/solutions/cloud-computing/1207639-comment-le-tour-de-france-est-entre-dans-une-nouvelle-dimension-avec-le-machine-learning/
https://www.journaldunet.com/ebusiness/publicite/1208328-ce-que-tous-les-secteurs-peuvent-tirer-du-revenue-management/
https://www.journaldunet.com/solutions/cloud-computing/1207016-mariadb-la-base-de-donnees-sql-qui-defie-oracle/
https://www.journaldunet.com/ebusiness/crm-marketing/1207284-data-video-et-temps-reel-place-a-l-hyperpersonnalisation-dans-vos-campagnes/
https://www.journaldunet.com/solutions/cloud-computing/1207135-vos-equipes-marketing-peinent-a-tirer-profit-du-big-data-voici-les-questions-a-se-poser/
https://www.journaldunet.com/solutions/cloud-computing/1207137-5-pieges-a-eviter-pour-analyser-d-importants-volumes-de-donnees/
https://www.journaldunet.com/solutions/cloud-computing/1207014-cloud-et-calcul-a-haute-performance

https://www.journaldunet.com/solutions/dsi/1192643-le-systeme-d-information-digital-doit-sacraliser-la-data/
https://www.journaldunet.com/solutions/dsi/1192580-comment-vous-premunir-face-aux-cyberattaques-et-autres-dangers-qui-menacent-vos-donnees/
https://www.journaldunet.com/solutions/dsi/1192320-presto-le-moteur-sql-distribue-qui-monte/
https://www.journaldunet.com/ebusiness/publicite/1192392-publicite-les-medias-numeriques-vont-ils-engloutir-les-medias-historiques/
https://www.journaldunet.com/solutions/dsi/1192220-le-big-data-est-mort-vive-la-donnee-centree-sur-la-valeur/
https://www.journaldunet.com/economie/finance/1191945-baptiste-beaume-covea-insurtech-big-data/
https://www.journaldunet.com/economie/industrie/1192383-une-strategie-de-supply-chain-bimodale-la-cle-pour-fournir-une-experience-client-fluide-et-sans-couture/
https://www.journaldunet.com/web-tech/1191410-a-t-on-encore-besoin-de-segmenter-les-clients-a-l-ere-du-big-data/
https://www.journaldunet.com/management/ressou

https://www.journaldunet.com/solutions/analytics/1180881-big-data-ou-l-injonction-paradoxale/
https://www.journaldunet.com/solutions/analytics/1181042-big-data-et-le-marketing-des-cohortes/
https://www.journaldunet.com/ebusiness/crm-marketing/1180624-la-realite-du-terrain-socle-de-l-avenir-numerique-des-entreprises/
https://www.journaldunet.com/solutions/cloud-computing/1180116-cloud-days-2016-outscale-organise-son-evenement-annuel-le-23-juin/
https://www.journaldunet.com/solutions/cloud-computing/1180135-comment-docker-est-sur-le-point-de-revolutionner-le-deploiement-du-big-data/
https://www.journaldunet.com/solutions/dsi/1179859-se-preparer-a-la-reglementation-eu-sur-la-protection-des-donnees/
https://www.journaldunet.com/solutions/dsi/1179602-le-big-data-du-centre-de-couts-au-centre-de-profits/
https://www.journaldunet.com/ebusiness/crm-marketing/1179672-reduire-la-fracture-multi-ecran/
https://www.journaldunet.com/solutions/dsi/1179367-27-des-projets-big-data-sont-deja-rentables/
h

...

https://www.journaldunet.com/ebusiness/crm-marketing/1143028-maximiser-vos-performances-digitales-grace-au-tms/
https://www.journaldunet.com/solutions/dsi/1142938-big-data-datastax-leve-106-millions-de-dollars/
https://www.journaldunet.com/ebusiness/le-net/1142943-etre-une-entreprise-numerique-ou-ne-plus-etre/
https://www.journaldunet.com/solutions/cloud-computing/1142940-le-big-data-a-de-beaux-jours-devant-lui/
https://www.journaldunet.com/solutions/cloud-computing/1142900-cloud-computing-pensez-chaine-de-valeur-globale-performance-et-capacite/
https://www.journaldunet.com/solutions/seo-referencement/1142758-le-referencement-naturel-pilote-par-les-donnees/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1142724-hausse-tres-modeste-pour-les-salaires-informatiques-en-2014/
https://www.journaldunet.com/management/ressources-humaines/1142657-avec-le-big-data-le-recrutement-entre-dans-une-nouvelle-ere/
https://www.journaldunet.com/solutions/dsi/1142090-pourquoi-les-editeu

https://www.journaldunet.com/solutions/cloud-computing/1136388-big-data-hortonworks-leve-100-millions-de-dollars/
https://www.journaldunet.com/solutions/analytics/1136359-paf-le-chien-booste-grace-a-hadoop-sur-microsoft-azure/
https://www.journaldunet.com/solutions/cloud-computing/1136376-cloud-google-lance-une-offensive-tarifaire-tous-azimuts/
https://www.journaldunet.com/solutions/analytics/1136352-analytics-et-big-data-dell-rachete-statsoft/
https://www.journaldunet.com/solutions/cloud-computing/1136312-cloudera-a-quoi-va-servir-sa-levee-de-fonds-de-160-millions-de-dollars/
https://www.journaldunet.com/ebusiness/commerce/1136229-transformer-les-donnees-de-type-big-data-en-donnees-intelligentes/
https://www.journaldunet.com/ebusiness/commerce/1136086-little-big-data-puisque-la-taille-ne-fait-pas-tout/
https://www.journaldunet.com/solutions/analytics/1136011-big-data-des-laboratoires-de-donnees-pour-reconcilier-dsi-et-operationnels/
https://www.journaldunet.com/solutions/seo-reference

https://www.journaldunet.com/solutions/dsi/1132039-cloud-et-big-data-champions-des-depenses-it-en-2014/
https://www.journaldunet.com/management/formation/1131878-le-big-data-entre-a-l-ecole/
https://www.journaldunet.com/solutions/dsi/1131480-big-data-cloud-et-mobilite-les-defis-du-dsi-face-aux-metiers/
https://www.journaldunet.com/solutions/cloud-computing/1131022-kinesis-l-offre-big-data-xxl-d-amazon/
https://www.journaldunet.com/web-tech/developpeur/1131023-watson-en-mode-cloud-est-ce-vraiment-realiste/
https://www.journaldunet.com/solutions/cloud-computing/1131011-orange-degaine-son-offre-big-data/
https://www.journaldunet.com/web-tech/developpeur/1130797-big-data-facebook-ouvre-les-sources-de-presto/
https://www.journaldunet.com/solutions/cloud-computing/1130579-big-data-le-cloud-d-amazon-supporte-desormais-hadoop-2-2/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1130400-at-internet-en-route-vers-le-big-data-avec-l-aide-de-l-etat/
https://www.journaldunet.com/s

https://www.journaldunet.com/solutions/dsi/1107923-big-data-paris-l-evenement-de-reference-du-big-data-les-3-et-4-avril/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1107715-les-salaires-informatiques-s-envolent-a-nouveau/
https://www.journaldunet.com/ebusiness/publicite/1107419-campagnes-de-liens-sponsorises-ce-qui-change-en-2013/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1106609-ibm-numero-1-des-brevets-pour-la-20e-annee-consecutive/
https://www.journaldunet.com/web-tech/start-up/1106423-les-entrepreneurs-doivent-apprendre-a-tirer-profit-des-donnees-des-services-qu-ils-developpent/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1104988-firehose-le-secret-de-mieux-en-mieux-garde-de-twitter/
https://www.journaldunet.com/solutions/cloud-computing/1104791-big-data-amazon-se-lance-dans-l-entrepot-de-donnees-dans-le-cloud/
https://www.journaldunet.com/solutions/reseau-social-d-entreprise/1104425-a-quoi-sert-watson-la-technolog

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,\n\n Avec plus de 2 600 milliards d...,"[\n, [\n Avec plus de 2 600 milliar...",2020-12-02,fr,\n Les graphes ...,https://www.journaldunet.com/economie/finance/...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Pascal Mary,"[""controller content"", ""cms journaldunet 14960..."
1,\n\n La mise en œuvre d'une politiq...,"[\n, [\n La mise en œuvre d'une pol...",2020-11-23,fr,\n La data resp...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Patrick Ottavi,"[""controller content"", ""cms journaldunet 14956..."
2,\n\n Des questions techniques relat...,"[\n, [\n Des questions techniques r...",2020-11-12,fr,\n La simplific...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Jonathan Gosselin,"[""controller content"", ""cms journaldunet 14951..."
3,"\n\n Certes, l'intuition, la percep...","[\n, [\n Certes, l'intuition, la pe...",2020-11-03,fr,\n Maîtrisez le...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Goulwen Pichot De Champfleury,"[""controller content"", ""cms journaldunet 14950..."
4,\n\n Les entreprises se retrouvent ...,"[\n, [\n Les entreprises se retrouv...",2020-10-28,fr,\n La dette lié...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,David Zydron,"[""controller content"", ""cms journaldunet 14949..."
...,...,...,...,...,...,...,...,...,...,...,...,...
806,\n\n Red Hat Storage Software Appli...,"[\n, [\n Red Hat Storage Software A...",2011-12-09,fr,\n Red Hat lanc...,https://www.journaldunet.com/solutions/reseau-...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Dominique Filippone,"[""controller content"", ""cms journaldunet 10958..."
807,\n\n La nouvelle solution de la div...,"[\n, [\n La nouvelle solution de la...",2011-12-08,fr,\n EMC se lance...,https://www.journaldunet.com/solutions/reseau-...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,La Rédaction,"[""controller content"", ""cms journaldunet 10957..."
808,\n\n La Business Intelligence fait ...,"[\n, [\n La Business Intelligence f...",2011-11-17,fr,\n Big data et ...,https://www.journaldunet.com/solutions/reseau-...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Dominique Filippone,"[""controller content"", ""cms journaldunet 10948..."
809,\n\n Présent depuis mai 2010 sur le...,"[\n, [\n Présent depuis mai 2010 su...",2011-10-24,fr,\n Après Micros...,https://www.journaldunet.com/solutions/reseau-...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,La Rédaction,"[""controller content"", ""cms journaldunet 10940..."


In [42]:
#Test on https://www.journaldunet.com/cybersecurite/list?page=1
url_pages = get_url_page_jdn_nb_pages("https://www.journaldunet.com/cybersecurite/list?page=1")
url_articles = []
for page in url_pages:
    url_articles.extend(get_url_article_jdn(page))
df_test = df_scrap
for article in url_articles:
    print(article)
    new_row = scraping(article)
    df_test = df_test.append(new_row, ignore_index=True)
df_test

https://www.journaldunet.com/solutions/dsi/1496707-stanislas-de-bentzmann-devoteam/
https://www.journaldunet.com/solutions/dsi/1496655-la-cybersecurite-est-une-question-informationnelle/
https://www.journaldunet.com/solutions/dsi/1496653-la-securite-du-cloud-post-pandemie-doit-demarrer-a-l-edge-avec-l-hyper-automatisation/
https://www.journaldunet.com/web-tech/cloud/1496645-les-10-previsions-technologiques-cloud-et-cybersecurite-pour-2021/
https://www.journaldunet.com/solutions/dsi/1496537-les-pme-et-les-tpe-doivent-s-armer-pour-affronter-les-ransomwares-as-a-service/
https://www.journaldunet.com/solutions/dsi/1496549-le-pouvoir-du-stockage-a-l-ere-de-l-industrie-4-0/
https://www.journaldunet.com/solutions/dsi/1496567-la-protection-des-donnees-un-sujet-prioritaire-par-ces-temps-incertains-3-tendances-qui-devraient-rythmer-l-annee-2021/
https://www.journaldunet.com/solutions/dsi/1496571-e-commercants-pas-de-joyeux-noel-sans-cybersecurite/
https://www.journaldunet.com/solutions/dsi/14965

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,\n\n Dans le sillage de la crise du...,"[\n, [\n Dans le sillage de la cris...",2021-01-07,fr,\nStanislas de Bentzmann (Devoteam)\n ...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Antoine Crochet-Damais,"[""controller content"", ""cms journaldunet 14967..."
1,\n\n Les remparts de la sécurité IT...,"[\n, [\n Les remparts de la sécurit...",2021-01-05,fr,\n La cybersécu...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Benjamin De Rose,"[""controller content"", ""cms journaldunet 14966..."
2,"\n\n Dans beaucoup d'entreprises, p...","[\n, [\n Dans beaucoup d'entreprise...",2021-01-05,fr,\n La sécurité ...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,David Shepherd,"[""controller content"", ""cms journaldunet 14966..."
3,"\n\n Rôle de l'IT, priorités, expér...","[\n, [\n Rôle de l'IT, priorités, e...",2021-01-04,fr,\n Les 10 prévi...,https://www.journaldunet.com/web-tech/cloud/14...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Didier Schreiber,"[""controller content"", ""cms journaldunet 14966..."
4,\n\n Avec la démocratisation des at...,"[\n, [\n Avec la démocratisation de...",2020-12-21,fr,\n Les PME et l...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,John Shier,"[""controller content"", ""cms journaldunet 14965..."
5,\n\n Les entreprises du secteur ind...,"[\n, [\n Les entreprises du secteur...",2020-12-21,fr,\n Le pouvoir d...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Sebastien Verger,"[""controller content"", ""cms journaldunet 14965..."
6,\n\n De l'automatisation par l'IA à...,"[\n, [\n De l'automatisation par l'...",2020-12-21,fr,\n Protection d...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Nicolas Combaret,"[""controller content"", ""cms journaldunet 14965..."
7,\n\n 73% des consommateurs prévoien...,"[\n, [\n 73% des consommateurs prév...",2020-12-21,fr,\n E-commerçant...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Yann Fralo,"[""controller content"", ""cms journaldunet 14965..."
8,\n\n Déjà victime de nombreux actes...,"[\n, [\n Déjà victime de nombreux a...",2020-12-21,fr,\n Les DDoS ne ...,https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Philippe Alcoy,"[""controller content"", ""cms journaldunet 14965..."
9,\n\n Le recours grandissant au e-co...,"[\n, [\n Le recours grandissant au ...",2020-12-18,fr,"\n Le CIAM, la ...",https://www.journaldunet.com/solutions/dsi/149...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Pierre Bar,"[""controller content"", ""cms journaldunet 14964..."


In [43]:
#Test on https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1
url_pages = get_url_page_jdn_nb_pages("https://www.journaldunet.com/ebusiness/le-net/fintech/list?page=1")
url_articles = []
for page in url_pages:
    url_articles.extend(get_url_article_jdn(page))
df_test = df_scrap
for article in url_articles:
    print(article)
    new_row = scraping(article)
    df_test = df_test.append(new_row, ignore_index=True)
df_test

https://www.journaldunet.com/patrimoine/guide-des-finances-personnelles/1210185-bitcoin-son-cours-s-envole-vers-les-40-000-dollars/
https://www.journaldunet.com/economie/finance/1496723-paiement-les-tendances-a-surveiller-et-les-incontournables-de-2021/
https://www.journaldunet.com/economie/finance/1496727-pennylane-leve-15-millions-d-euros-pour-creer-un-appstore-des-services-financiers/
https://www.journaldunet.com/economie/finance/1487516-galaxie-fintech/
https://www.journaldunet.com/patrimoine/guide-des-finances-personnelles/1489437-ethereum-le-cours-tutoie-les-1-000-dollars/
https://www.journaldunet.com/management/direction-generale/1496633-brexit-et-rgpd-les-entreprises-beneficieront-de-6-mois-pour-les-transferts-de-donnees-personnelles/
https://www.journaldunet.com/solutions/dsi/1496607-transformation-digitale-des-assureurs-le-role-cle-des-apis/
https://www.journaldunet.com/economie/finance/1496593-chargeback-la-necessite-de-limiter-les-risques-de-fraude/
https://www.journaldunet

https://www.journaldunet.com/economie/finance/1494517-credit-virement-scoring-la-nouvelle-vague-de-l-open-banking/
https://www.journaldunet.com/economie/finance/1494445-joko-leve-10-millions-d-euros-pour-son-appli-de-cashback/
https://www.journaldunet.com/economie/finance/1494469-fintech-grands-groupes-privilegions-l-ecosysteme-chacun-y-gagnera/
https://www.journaldunet.com/economie/finance/1494307-fintech-revolution-2020-fait-le-plein-le-15-octobre-prochain-a-station-f/
https://www.journaldunet.com/ebusiness/commerce/1494233-succes-des-e-casinos-en-periode-de-confinement-quels-sont-les-principaux-enjeux/
https://www.journaldunet.com/economie/finance/1494303-akur8-elue-insurtech-de-l-annee-2020/
https://www.journaldunet.com/economie/finance/1494273-alain-clot-france-fintech/
https://www.journaldunet.com/economie/finance/1494219-defi-derriere-l-engouement-fulgurant-de-cet-ete/
https://www.journaldunet.com/economie/finance/1494199-regime-europeen-pour-les-crypto-monnaies-sitot-publie-sit

https://www.journaldunet.com/economie/finance/1491679-grace-au-confinement-le-credit-conso-finalise-sa-transition-numerique/
https://www.journaldunet.com/economie/finance/1491609-charles-egly-younited-credit/
https://www.journaldunet.com/economie/finance/1491621-post-crise-covid-19-la-banque-va-necessairement-faire-evoluer-sa-maniere-d-accompagner-les-commercants/
https://www.journaldunet.com/economie/finance/1491587-bitcoin-une-protection-contre-l-inflation-en-2020/
https://www.journaldunet.com/economie/finance/1489739-pour-mieux-seduire-les-entreprises-l-epargne-salariale-revoit-ses-plans/
https://www.journaldunet.com/economie/finance/1491437-derriere-la-crise-du-covid-19-la-place-des-seniors-en-question/
https://www.journaldunet.com/economie/finance/1491405-cedric-malengreau-arkea/
https://www.journaldunet.com/economie/finance/1491423-authentification-forte-la-pression-monte-pour-obtenir-un-report/
https://www.journaldunet.com/economie/finance/1491365-infographie-ces-fintech-qui-viv

https://www.journaldunet.com/economie/finance/1488597-renaud-laplanche-upgrade/
https://www.journaldunet.com/economie/finance/1488574-le-financement-obligataire-une-opportunite-ignoree-pour-la-croissance-de-nos-pme/
https://www.journaldunet.com/economie/finance/1488557-l-avenement-du-social-trading/
https://www.journaldunet.com/economie/finance/1488390-pieter-van-der-does-adyen/
https://www.journaldunet.com/economie/finance/1488525-quelles-perspectives-pour-la-fintech-et-l-insurtech-en-2020/
https://www.journaldunet.com/economie/finance/1488517-vers-une-gestion-decentralisee-de-l-engagement-de-depenses/
https://www.journaldunet.com/economie/finance/1488353-gcollect-moment-rubypayeur-et-upflow-quatre-pepites-francaises-qui-luttent-contre-les-impayes/
https://www.journaldunet.com/economie/finance/1488347-quand-la-france-et-l-allemagne-avancent-d-un-meme-pas-sur-les-crypto-monnaies/
https://www.journaldunet.com/economie/finance/1488223-denis-beau-banque-de-france/
https://www.journaldunet

https://www.journaldunet.com/economie/finance/1446591-comment-alibaba-bouleverse-la-finance-en-chine/
https://www.journaldunet.com/economie/finance/1207903-qui-sont-les-assurtech-francaises-2019/
https://www.journaldunet.com/economie/finance/1446157-cybersecurite-les-banques-doivent-miser-sur-la-detection-et-la-prevention/
https://www.journaldunet.com/economie/finance/1446518-et-si-la-blockchain-changeait-la-conformite-financiere/
https://www.journaldunet.com/economie/finance/1446523-un-an-d-investissement-dans-la-fintech-europeenne-5-lecons-a-retenir/
https://www.journaldunet.com/economie/finance/1446075-rebecca-loevenguth-western-union/
https://www.journaldunet.com/economie/finance/1446057-le-risque-operationnel-l-enjeu-reglementaire-des-neobanques/
https://www.journaldunet.com/economie/finance/1446151-la-cyber-assurance-une-necessite-et-un-marche-extremement-porteur/
https://www.journaldunet.com/economie/finance/1442504-paiement-comment-paylib-veut-faire-mieux-qu-amazon/
https://www

...

https://www.journaldunet.com/economie/finance/1194363-l-imbroglio-reglementaire-des-fintechs-wansquare/
https://www.journaldunet.com/patrimoine/finances-personnelles/1194313-francois-perol-bpce-orange-bank/
https://www.journaldunet.com/economie/finance/1194288-plus-de-450-api-sont-referencees-sur-notre-plateforme-btob/
https://www.journaldunet.com/economie/finance/1194292-banque-et-assurance-comment-stocker-et-proteger-ses-donnees-efficacement/
https://www.journaldunet.com/economie/finance/1194117-banques-et-assurances-investissent-dans-la-donnee-en-2017/
https://www.journaldunet.com/economie/finance/1194139-younited-credit-approche-de-son-objectif-de-leader-europeen-wansquare/
https://www.journaldunet.com/economie/finance/1194109-open-banking-les-menaces-sur-les-banques/
https://www.journaldunet.com/economie/finance/1194359-la-blockchain-isation-fin-annoncee-de-l-uberisation/
https://www.journaldunet.com/economie/finance/1194038-facebook-marche-du-crowdfunding/
https://www.journaldune

https://www.journaldunet.com/economie/finance/1187624-la-gestion-des-comptes-bancaires-de-la-digitalisation-a-la-revolution-de-l-open-banking/
https://www.journaldunet.com/economie/finance/1187632-la-mutualisation-des-moyens-de-traitement-des-cheques-une-priorite-qui-s-impose-pour-les-banques/
https://www.journaldunet.com/economie/finance/1187039-statista-services-bancaires-mobiles-statista/
https://www.journaldunet.com/economie/finance/1187227-les-neo-banques-ne-sont-pas-pres-d-etre-rentables/
https://www.journaldunet.com/economie/finance/1187160-blockchain-comment-ibm-a-mis-la-main-sur-le-projet-hyperledger/
https://www.journaldunet.com/economie/finance/1186593-younited-credit-siege/
https://www.journaldunet.com/economie/finance/1186771-confidentiel-la-fintech-d-expertise-comptable-smallbusinessact-leve-1-1-million-d-euros/
https://www.journaldunet.com/economie/finance/1186571-ibanfirst-leve-5-millions-d-euros-et-devient-une-neo-banque-pour-pme/
https://www.journaldunet.com/economie/

https://www.journaldunet.com/economie/finance/1177962-forum-economique-mondial-les-6-plus-grands-risques-de-la-fintech/
https://www.journaldunet.com/economie/finance/1178494-monnaie-virtuelle-statista/
https://www.journaldunet.com/economie/finance/1178681-le-rachat-de-depopass-va-completer-l-offre-de-paiement-de-s-money/
https://www.journaldunet.com/economie/finance/1180217-barometre-du-crowdfunding-avril-2016/
https://www.journaldunet.com/economie/finance/1178252-l-agregateur-nouvel-acteur-majeur-du-secteur-bancaire/
https://www.journaldunet.com/economie/finance/1177954-ing-direct-start-up/
https://www.journaldunet.com/economie/finance/1177833-comment-les-regtech-veulent-rendre-aux-banques-leur-agilite/
https://www.journaldunet.com/patrimoine/finances-personnelles/1178878-conseillers-patrimoniaux-et-transformation-digitale-vers-une-evolution-des-metiers/
https://www.journaldunet.com/economie/finance/1177796-cdo-bnp-paribas-investment-partners/
https://www.journaldunet.com/economie/fin

https://www.journaldunet.com/web-tech/start-up/1152917-allianz-france-et-idinvest-creent-un-fonds-pour-investir-sur-smartangels/
https://www.journaldunet.com/web-tech/start-up/1152818-barclays-accelerator-l-accelerateur-londonien-dedie-aux-fintech/
https://www.journaldunet.com/web-tech/start-up/1152758-fintech-gocardless-se-lance-en-france-pour-simplifier-le-prelevement-bancaire/
https://www.journaldunet.com/ebusiness/le-net/1152321-les-acsel-du-numerique-2015-couronnent-six-laureats/
https://www.journaldunet.com/web-tech/start-up/1152080-lendix-plateforme-de-prets-aux-pme-recueille-28-millions-d-euros-de-financement/
https://www.journaldunet.com/ebusiness/le-net/1151655-fintech-boursorama-rachete-fiduceo-un-outil-de-gestion-de-finances-personnelles/
https://www.journaldunet.com/web-tech/start-up/1151488-confidentiel-unilend-leve-8-millions-d-euros-pour-preter-aux-pme/
https://www.journaldunet.com/web-tech/start-up/1150618-axa-cree-un-fonds-de-capital-risque-de-200-millions-et-un-accel

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,art_img,art_auth,art_tag
0,\n\n\n\n\n [CRYPTOMONNAIES] Dep...,"[\n, [\n, [\n, <img alt=""Bitcoin : son cours s...",2021-01-07,fr,\n Bitcoin : so...,https://www.journaldunet.com/patrimoine/guide-...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/NwasJE9WCK3gicc...,Charlie Perreau,"[""controller content"", ""cms journaldunet 12101..."
1,\n\n Accélération vers le e-commerc...,"[\n, [\n Accélération vers le e-com...",2021-01-07,fr,\n Paiement : l...,https://www.journaldunet.com/economie/finance/...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Christophe Mariette,"[""controller content"", ""cms journaldunet 14967..."
2,\n\n\n\n\n Un an après son lanc...,"[\n, [\n, [\n, <img alt=""Pennylane lève 15 mil...",2021-01-07,fr,\n Pennylane lè...,https://www.journaldunet.com/economie/finance/...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/dlWCQf8ImU4joWs...,Charlie Perreau,"[""controller content"", ""cms journaldunet 14967..."
3,\n\n\n\n\n Nouveauté de cette s...,"[\n, [\n, [\n, <img alt=""Exclusif : parcourez ...",2021-01-04,fr,\n Exclusif : p...,https://www.journaldunet.com/economie/finance/...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/W2iAnDkI3liexp2...,Charlie Perreau,"[""controller content"", ""cms journaldunet 14875..."
4,\n\n\n\n\n [ETHEREUM] La deuxiè...,"[\n, [\n, [\n, <img alt=""Ethereum : le cours t...",2021-01-04,fr,\n Ethereum : l...,https://www.journaldunet.com/patrimoine/guide-...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/6vfAHCc1p_iDh8P...,Charlie Perreau,"[""controller content"", ""cms journaldunet 14894..."
...,...,...,...,...,...,...,...,...,...,...,...,...
968,\n\n Mise en faillite de plateforme...,"[\n, [\n Mise en faillite de platef...",2014-03-07,fr,\nPhilippe Rodriguez (Bitcoin France)\n ...,https://www.journaldunet.com/ebusiness/le-net/...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Aude Fredouelle,"[""controller content"", ""cms journaldunet 11357..."
969,\n\n Cofondateur de Paymium (Bitcoi...,"[\n, [\n Cofondateur de Paymium (Bi...",2014-03-03,fr,\nPierre Noizat (Paymium)\n ...,https://www.journaldunet.com/ebusiness/le-net/...,journal du net,xpath_source,https://www.journaldunet.com/,no_data,Frantz Grenier,"[""controller content"", ""cms journaldunet 11354..."
970,\n\n\n\n\n Dans un entrepôt ind...,"[\n, [\n, [\n, <img alt=""En images : un entrep...",2013-12-06,fr,\n En images : ...,https://www.journaldunet.com/ebusiness/le-net/...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/MPZ3MjVYDB_kWx-...,Aude Fredouelle,"[""controller content"", ""cms journaldunet 11321..."
971,\n\n\n\n\n De nouveaux organism...,"[\n, [\n, [\n, <img alt=""Attention à vos amis ...",2013-09-27,fr,\n Attention à ...,https://www.journaldunet.com/ebusiness/le-net/...,journal du net,xpath_source,https://www.journaldunet.com/,https://img-0.journaldunet.com/kQRuQ4mIDAbDVCW...,Aude Fredouelle,"[""controller content"", ""cms journaldunet 11278..."
